# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2023spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/15lGUmT8NpLGtoxRllRWCJyQEjhR1Idcei63YHsDckPE/edit#slide=id.g21fff4e9af6_0_13)　Kaggle: [Link](https://www.kaggle.com/competitions/ml2023spring-hw7/host/sandbox-submissions)　Data: [Link](https://drive.google.com/file/d/1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7/view?usp=sharing)




# colab code

# # Reference link （all）
For specific location references, see the markdown and comments in the code below.

https://huggingface.co/luhua/chinese_pretrain_mrc_macbert_large


https://blog.csdn.net/qq_42994201/article/details/121442992


https://blog.csdn.net/weixin_42369818/article/details/124835855?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168221931016800226541667%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168221931016800226541667&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-124835855-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw7&spm=1018.2226.3001.4187


https://blog.csdn.net/orangerfun/article/details/120400247

https://gitee.com/yuys0602/NER-BERT-pytorch/

# Prerequisites

## Download Dataset

In [1]:
# download link 1
# !gdown --id '1TjoBdNlGBhP_J9C66MOY7ILIrydm7ZCS' --output hw7_data.zip

# download link 2 (if above link failed)
# !gdown --id '1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7' --output hw7_data.zip

# download link 3 (if above link failed)
!gdown --id '1k2BfGrvhk8QRnr9Xvb04oPIKDr1uWFpa' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1k2BfGrvhk8QRnr9Xvb04oPIKDr1uWFpa
To: /content/hw7_data.zip
100% 12.1M/12.1M [00:00<00:00, 191MB/s]
Archive:  hw7_data.zip
  inflating: hw7_train.json          
  inflating: hw7_test.json           
  inflating: hw7_dev.json            
  inflating: hw7_in-context-learning-examples.json  
Sun Apr 23 05:34:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|          

## Install packages

Documentation for the toolkit: 
*   https://huggingface.co/transformers/
*   https://huggingface.co/docs/accelerate/index



In [2]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.26.1
!pip install accelerate==0.16.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Kaggle (Fine-tuning)

## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Import Packages

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW

from transformers import BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(1370)

## Load Model and Tokenizer




 

# model reference
https://huggingface.co/luhua/chinese_pretrain_mrc_macbert_large

In [4]:
#from transformers import (
#  AutoTokenizer,
#  AutoModelForQuestionAnswering,
#)

#model = AutoModelForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
#tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

model_name = "luhua/chinese_pretrain_mrc_macbert_large"
#use model luhua/chinese_pretrain_mrc_macbert_large
model = BertForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = BertTokenizerFast.from_pretrained(model_name)
#reference https://huggingface.co/luhua/chinese_pretrain_mrc_macbert_large

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26918 QA pairs
- Dev set: 2863  QA pairs
- Test set: 3524  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [5]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [6]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset

In [7]:
train_batch_size = 8
doc_stride = 16

class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60
        self.max_paragraph_len = 150
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 32

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            #mid = (answer_start_token + answer_end_token) // 2
            #paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            #paragraph_end = paragraph_start + self.max_paragraph_len

            start_min = max(0, answer_end_token - self.max_paragraph_len + 1)
            start_max = min(answer_start_token, len(tokenized_paragraph) - self.max_paragraph_len)
            start_max = max(start_min, start_max)
            paragraph_start = random.randint(start_min, start_max + 1)
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation
postprocessing
https://blog.csdn.net/qq_42994201/article/details/121442992

unk handle
https://blog.csdn.net/weixin_42369818/article/details/124835855?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168221931016800226541667%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168221931016800226541667&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-124835855-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw7&spm=1018.2226.3001.4187

In [8]:
def evaluate(data, output, doc_stride=doc_stride, token_type_ids=None, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        token_type_id = data[1][0][k].detach().cpu().numpy()
        #[CLS] + [question] + [SEP] + [paragraph] + [SEP]
        paragraph_start = token_type_id.argmax()
        paragraph_end = len(token_type_id) - 1 - token_type_id[::-1].argmax() - 1

        #postprocessing ：https://blog.csdn.net/qq_42994201/article/details/121442992
        if start_index > end_index or start_index < paragraph_start or end_index > paragraph_end:
            continue
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

            # 找到tokenized paragraph中对应的位置
            origin_start = start_index + k * doc_stride - paragraph_start
            origin_end = end_index + k * doc_stride - paragraph_start
    
    answer = answer.replace(' ', '')
    
    if '[UNK]' in answer:
        print('发现 [UNK]，这表明有文字无法编码, 使用原始文本')
        #print("Paragraph:", paragraph)
        #print("Paragraph:", paragraph_tokenized.tokens)
        print('--直接解码预测:', answer)
        #找到原始文本中对应的位置
        raw_start = paragraph_tokenized.token_to_chars(origin_start)[0]
        raw_end = paragraph_tokenized.token_to_chars(origin_end)[1]
        answer = paragraph[raw_start:raw_end]
        print('--原始文本预测:',answer)
    #unk handle
    #reference link： https://blog.csdn.net/weixin_42369818/article/details/124835855?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168221931016800226541667%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168221931016800226541667&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-124835855-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw7&spm=1018.2226.3001.4187
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    #return answer.replace(' ','')

## Training
#reference link

https://blog.csdn.net/orangerfun/article/details/120400247

https://gitee.com/yuys0602/NER-BERT-pytorch/

In [9]:
from accelerate import Accelerator

# hyperparameters
num_epoch = 1
validation = False
logging_step = 100
total_steps = num_epoch * len(train_loader)
learning_rate = 1e-6
acc_steps = 1
optimizer = AdamW(model.parameters(), lr=learning_rate)
train_batch_size = 8

#learning rate reference link: https://blog.csdn.net/orangerfun/article/details/120400247
from transformers import get_linear_schedule_with_warmup
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=total_steps//acc_steps)

#### TODO: gradient_accumulation (optional)####
# Note: train_batch_size * gradient_accumulation_steps = effective batch size
# If CUDA out of memory, you can make train_batch_size lower and gradient_accumulation_steps upper
# Doc: https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation
gradient_accumulation_steps = 16

# dataloader
# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)


# Change "fp16_training" to True to support automatic mixed 
# precision training (fp16)	
fp16_training = True
if fp16_training:    
    accelerator = Accelerator(mixed_precision="fp16")
else:
    accelerator = Accelerator()


# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

#no validation： https://gitee.com/yuys0602/NER-BERT-pytorch/
if not validation:
    dev_set = QA_Dataset("train", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
    train_set = torch.utils.data.ConcatDataset([train_set, dev_set])
    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    optimizer.zero_grad()
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
           
        train_loss += output.loss
        
        accelerator.backward(output.loss)
        
        step += 1
        if step % acc_steps == 0:
            #grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
        
        ##### TODO: Apply linear learning rate decay #####

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            lr = optimizer.state_dict()['param_groups'][0]['lr']
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3723 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 2.179, acc = 0.421
Epoch 1 | Step 200 | loss = 1.228, acc = 0.577
Epoch 1 | Step 300 | loss = 0.957, acc = 0.627
Epoch 1 | Step 400 | loss = 0.851, acc = 0.640
Epoch 1 | Step 500 | loss = 0.810, acc = 0.700
Epoch 1 | Step 600 | loss = 0.809, acc = 0.644
Epoch 1 | Step 700 | loss = 0.752, acc = 0.702
Epoch 1 | Step 800 | loss = 0.716, acc = 0.689
Epoch 1 | Step 900 | loss = 0.739, acc = 0.680
Epoch 1 | Step 1000 | loss = 0.702, acc = 0.701
Epoch 1 | Step 1100 | loss = 0.637, acc = 0.717
Epoch 1 | Step 1200 | loss = 0.682, acc = 0.719
Epoch 1 | Step 1300 | loss = 0.713, acc = 0.716
Epoch 1 | Step 1400 | loss = 0.660, acc = 0.744
Epoch 1 | Step 1500 | loss = 0.628, acc = 0.748
Epoch 1 | Step 1600 | loss = 0.657, acc = 0.730
Epoch 1 | Step 1700 | loss = 0.620, acc = 0.750
Epoch 1 | Step 1800 | loss = 0.665, acc = 0.743
Epoch 1 | Step 1900 | loss = 0.731, acc = 0.700
Epoch 1 | Step 2000 | loss = 0.560, acc = 0.766
Epoch 1 | Step 2100 | loss = 0.597, acc = 0.750
E

## Testing

# model reference
https://huggingface.co/luhua/chinese_pretrain_mrc_macbert_large

In [ ]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    #for data in tqdm(test_loader):
    #new model need： https://huggingface.co/luhua/chinese_pretrain_mrc_macbert_large
    for i, data in enumerate(tqdm(test_loader)):
        #output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       #attention_mask=data[2].squeeze(dim=0).to(device))
        #result.append(evaluate(data, output))
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, doc_stride=doc_stride, paragraph=test_paragraphs[test_questions[i]["paragraph_id"]],
                               paragraph_tokenized=test_paragraphs_tokenized[test_questions[i]["paragraph_id"]]))

result_file = "result.csv"
with open(result_file, 'w') as f:	
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 回[UNK]
--原始文本预测: 位年
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]
--原始文本预测: 輸
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 回[UNK]汗國
--原始文本预测: 回鶻汗國
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 白[UNK]紀中期
--原始文本预测: 白堊紀中期
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]大壩
--原始文本预测: 壩一般
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]
--原始文本预测: 下
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 李[UNK]
--原始文本预测: 李勣
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]
--原始文本预测: 使
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 久彌宮妃[UNK]子
--原始文本预测: 在戰爭中長大
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 《阿[UNK]婆吠陀》
--原始文本预测: 》、《娑摩吠陀
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 姚[UNK]
--原始文本预测: 北復
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]-850型
--原始文本预测: 了三架本
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 分[UNK]期淺灌，幼穗分化開始間隙灌水
--原始文本预测: 幼穗形成時，還有抽穗開花期加強水
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK][UNK]自動步槍系列
--原始文本预测: 。MP5系列的發明把
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]水之戰
--原始文本预测: 潰，北方
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 金[UNK]
--原始文

# GradeScope - Question 2 (In-context learning)

### In-context learning
The example prompt is :
```
請從最後一篇的文章中找出最後一個問題的答案：
文章：<文章1 內容>
問題：<問題1 敘述>
答案：<答案1>
...
文章：<文章n 內容>
問題：<問題n 敘述>
答案：
```

In [ ]:
import torch
import random  
import numpy as np

# To avoid CUDA_OUT_OF_MEMORY
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Fix random seed for reproducibility
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# You can try model with different size
# When using Colab or Kaggle, models with more than 2 billions parameters may 
# run out of memory
tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-1.7B")

In [ ]:
# To clean model output. If you try different prompts, you may have to fix 
# this function on your own
def clean_text(text):
    # Note: When you use unilingual model, the colon may become fullwidth
    text = text.split("答案:")[-1]
    text = text.split(" ")[0]
    return text

In [ ]:
import random
import json

with open("hw7_in-context-learning-examples.json", "r") as f: 
    test = json.load(f)

# K-shot learning 
# Give model K examples to make it achieve better accuracy 
# Note: (1) When K >= 4, CUDA_OUT_OFF_MEMORY may occur.
#       (2) The maximum input length of XGLM is 2048
K = 2

question_ids = [qa["id"] for qa in test["questions"]]

with open("in-context-learning-result.txt", "w") as f:
    print("ID,Ground-Truth,Prediction", file = f)
    with torch.no_grad():
        for idx, qa in enumerate(test["questions"]):
            # You can try different prompts
            #prompt = "請從最後一篇的文章中找出最後一個問題的答案\n" #normal prompt
            prompt = "答案限制在兩句話內，請根據最後一篇文章回答最後一個問題\n"  # Sentence structure restriction
            #prompt = "答案限制在10個字內，請根據最後一篇文章回答最後一個問題\n"  # WORD LIMIT
            exist_question_indexs = [question_ids.index(qa["id"])]

            # K-shot learning: give the model K examples with answers
            for i in range(K):
                question_index = question_ids.index(qa["id"])
                while(question_index in exist_question_indexs): 
                    question_index = random.randint(0, len(question_ids) - 1)
                exist_question_indexs.append(question_index)    
                paragraph_id = test["questions"][question_index]["paragraph_id"]
                prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
                prompt += f'問題：{test["questions"][question_index]["question_text"]}\n'
                prompt += f'答案：{test["questions"][question_index]["answer_text"]}\n'

            # The final one question without answer
            paragraph_id = qa["paragraph_id"]
            prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
            prompt += f'問題：{qa["question_text"]}\n'
            prompt += f'答案：'
            
            inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt") 
            sample = model.generate(**inputs, max_new_tokens = 20)
            text = tokenizer.decode(sample[0], skip_special_tokens=True)

            # Note: You can delete this line to see what will happen
            text = clean_text(text)
            
            print(prompt)
            print(f'正確答案: {qa["answer_text"]}')
            print(f'模型輸出: {text}')
            print()

            print(f"{idx},{qa['answer_text']},{text}", file = f)